# evaluation.py
This module computes the fitness of a particular candidate solution based on hard constraints and soft constraints.

In [1]:
import sys
import config

Main driver of module. Calculates a fitness value for a candidate solution based on the number of conflicts where higher is better (max 100). Returns an integer.

In [2]:
def calc_fitness(candidate_solution):
    fitness = 100
    hc = hard_constraints(candidate_solution)
    sc = soft_constraints(candidate_solution)

    return [max(fitness - hc - sc,1),sc]


Checks a candidate solution for hard constraints, returns the number of violated constraints. Hard constraints
include room capacity conflicts, room schedule conflicts and prof conflicts. 

In [3]:
def hard_constraints(candidate_solution):
    hc = 0
    hc += check_rooms(candidate_solution)
    hc += check_profs(candidate_solution)
    hc += check_capacity(candidate_solution)
    return hc

Checks a candidate solution for soft constraints, returns the number of violated constraints. Soft constraints include professors teaching back-to-back, conescutive courses scheduled back-to-back, and courses in the same year and faculty being scheduled in the same time slot. 

In [4]:
def soft_constraints(candidate_solution):
    sc = 0
    sc += check_prof_back_to_back(candidate_solution)
    sc += check_course_back_to_back(candidate_solution)
    sc += check_course_years(candidate_solution)
    return sc 

Checks to make sure there are no duplicate room and section times (ie. no two classes are scheduled in the same room at the same time). Returns number of conflicts found.

In [5]:
def check_rooms(candidate_solution):
    rooms = []
    returnval = 0
    for course, attrs in candidate_solution.items():
        if course != "Fitness":
            sections_rooms = [attrs["time"], attrs["room"]]
            if sections_rooms in rooms:
                returnval += 5
            rooms.append(sections_rooms)
    return returnval

Checks to make sure there are no duplicate prof and section times (ie. no prof can be in two places at once). Returns number of conflicts found.

In [6]:
def check_profs(candidate_solution):
    profs = []
    returnval = 0
    for course, attrs in candidate_solution.items():
        if course != "Fitness":
            sections_profs = [attrs["time"], attrs["prof"]]
            if sections_profs in profs:
                returnval += 5
            profs.append(sections_profs)           
    return returnval

Checks to make sure there a classes enrollment can fit in the selected room. Returns number of conflicts.

In [7]:
def check_capacity(candidate_solution):
    room_capacities = list(config.config_rooms().values())
    enrolments = config.config_courses()
    returnval = 0
    for course, attrs in candidate_solution.items():
        if course != "Fitness":
            class_enrolment = enrolments[course]["Enrolment"] 
            room = attrs["room"]
            room_cap = room_capacities[room]["Capacity"]
            if class_enrolment > room_cap:
                returnval += 5
    return returnval

Checks soft constraint of a prof having to teach a class back to back. Returns number of violated constraints. 

In [8]:
def check_prof_back_to_back(candidate_solution):
    returnval = 0
    prof_dict = {}
    for course, data in candidate_solution.items():
        if course == "Fitness":
            continue
        if data["prof"] not in prof_dict.keys():
            prof_dict[data["prof"]] = [int(data["time"])]
        else:
            prof_dict[data["prof"]].append(int(data["time"]))
        sc = 0
        for time1 in prof_dict[data["prof"]]:
            for time2 in prof_dict[data["prof"]]:
                if time1 != time2:
                    if ((time1 + 1) == time2) or ((time1 - 1) == time2):
                        sc += 1
        returnval += (sc//2)
    return returnval

Checks soft constraint of consecutive courses being scheduled back to back. Returns number of violated constraints.

In [9]:
def check_course_back_to_back(candidate_solution):
    returnval = 0
    for course1, data1 in candidate_solution.items():
        if course1 != "Fitness":
            for course2, data2 in candidate_solution.items():
                if course2 != "Fitness":
                    if (course1[0:4] == course2[0:4]) and (course1 != course2):
                        if ((int(course1[5:8]) + 1) == int(course2[5:8])) or \
                           ((int(course1[5:8]) - 1) == int(course2[5:8])):
                            if ((int(data1["time"]) + 1) == int(data2["time"])) or \
                               ((int(data1["time"]) - 1) == int(data2["time"])):
                                returnval += 1
    return (returnval//2)

Checks soft constraint of courses of the same year scheduled back to back. Returns number of violated constraints.

In [10]:
def check_course_years(candidate_solution):
    returnval = 0
    course_dict = {}
    for course, data in candidate_solution.items():
        if course == "Fitness":
            continue
        if course[0:6] not in course_dict.keys():
            course_dict[course[0:6]] = [int(data["time"])]
        else:
            if int(data["time"]) in course_dict[course[0:6]]:
                returnval += 1
            course_dict[course[0:6]].append(int(data["time"]))
    return returnval

Demonstration purposes

In [11]:
sample_solution = {'CISC 101': {'time': 7, 'room': 82, 'prof': 'Hu'}, 'CISC 102': {'time': 8, 'room': 98, 'prof': 'Blostein'}, 'CISC 103': {'time': 42, 'room': 95, 'prof': 'Dove'}, 'CISC 104': {'time': 19, 'room': 52, 'prof': 'Dawes'}, 'CISC 105': {'time': 0, 'room': 10, 'prof': 'Powley'}, 'CISC 201': {'time': 27, 'room': 23, 'prof': 'Graham'}, 'CISC 202': {'time': 21, 'room': 7, 'prof': 'Hu'}, 'CISC 203': {'time': 22, 'room': 74, 'prof': 'Cordy'}, 'CISC 204': {'time': 42, 'room': 93, 'prof': 'Lamb'}, 'CISC 205': {'time': 34, 'room': 39, 'prof': 'Graham'}, 'CISC 301': {'time': 7, 'room': 18, 'prof': 'Dove'}, 'CISC 302': {'time': 44, 'room': 16, 'prof': 'Dove'}, 'CISC 303': {'time': 31, 'room': 72, 'prof': 'Powley'}, 'CISC 304': {'time': 37, 'room': 12, 'prof': 'Blostein'}, 'CISC 305': {'time': 22, 'room': 15, 'prof': 'Rappaport'}, 'CISC 401': {'time': 14, 'room': 6, 'prof': 'Dawes'}, 'CISC 402': {'time': 33, 'room': 64, 'prof': 'Lamb'}, 'CISC 403': {'time': 20, 'room': 83, 'prof': 'Hu'}, 'CISC 404': {'time': 8, 'room': 68, 'prof': 'Graham'}, 'CISC 405': {'time': 33, 'room': 82, 'prof': 'Blostein'}, 'MATH 101': {'time': 41, 'room': 63, 'prof': 'Day'}, 'MATH 102': {'time': 1, 'room': 15, 'prof': 'Ableson'}, 'MATH 103': {'time': 17, 'room': 66, 'prof': 'Ableson'}, 'MATH 104': {'time': 40, 'room': 71, 'prof': 'Takahara'}, 'MATH 105': {'time': 38, 'room': 94, 'prof': 'Mingo'}, 'MATH 201': {'time': 44, 'room': 29, 'prof': 'Alajaji'}, 'MATH 202': {'time': 37, 'room': 27, 'prof': 'Kyle'}, 'MATH 203': {'time': 23, 'room': 112, 'prof': 'Kyle'}, 'MATH 204': {'time': 44, 'room': 42, 'prof': 'Takahara'}, 'MATH 205': {'time': 17, 'room': 22, 'prof': 'Cellarosi'}, 'MATH 301': {'time': 33, 'room': 42, 'prof': 'Mingo'}, 'MATH 302': {'time': 25, 'room': 105, 'prof': 'Day'}, 'MATH 303': {'time': 3, 'room': 113, 'prof': 'Lewis'}, 'MATH 304': {'time': 3, 'room': 84, 'prof': 'Takahara'}, 'MATH 305': {'time': 35, 'room': 34, 'prof': 'Roth'}, 'MATH 401': {'time': 26, 'room': 42, 'prof': 'Alajaji'}, 'MATH 402': {'time': 23, 'room': 25, 'prof': 'Li'}, 'MATH 403': {'time': 18, 'room': 94, 'prof': 'Roth'}, 'MATH 404': {'time': 27, 'room': 46, 'prof': 'Kani'}, 'MATH 405': {'time': 41, 'room': 104, 'prof': 'Mingo'}, 'ENGL 101': {'time': 43, 'room': 17, 'prof': 'Brooke'}, 'ENGL 102': {'time': 0, 'room': 58, 'prof': 'Fanning'}, 'ENGL 103': {'time': 20, 'room': 109, 'prof': 'Fanning'}, 'ENGL 104': {'time': 27, 'room': 107, 'prof': 'McIntire'}, 'ENGL 105': {'time': 2, 'room': 31, 'prof': 'Moriah'}, 'ENGL 201': {'time': 6, 'room': 54, 'prof': 'Ritchie'}, 'ENGL 202': {'time': 15, 'room': 57, 'prof': 'Straker'}, 'ENGL 203': {'time': 36, 'room': 85, 'prof': 'Morrison'}, 'ENGL 204': {'time': 26, 'room': 63, 'prof': 'Pierce'}, 'ENGL 205': {'time': 11, 'room': 27, 'prof': 'Smart'}, 'ENGL 301': {'time': 2, 'room': 44, 'prof': 'Brooke'}, 'ENGL 302': {'time': 25, 'room': 93, 'prof': 'Peacocke'}, 'ENGL 303': {'time': 19, 'room': 90, 'prof': 'Moriah'}, 'ENGL 304': {'time': 36, 'room': 78, 'prof': 'Wallace'}, 'ENGL 305': {'time': 39, 'room': 65, 'prof': 'Fanning'}, 'ENGL 401': {'time': 8, 'room': 103, 'prof': 'May'}, 'ENGL 402': {'time': 13, 'room': 101, 'prof': 'May'}, 'ENGL 403': {'time': 30, 'room': 54, 'prof': 'Ritchie'}, 'ENGL 404': {'time': 38, 'room': 48, 'prof': 'Pierce'}, 'ENGL 405': {'time': 15, 'room': 14, 'prof': 'Fanning'}, 'PHYS 101': {'time': 40, 'room': 61, 'prof': 'Morelli'}, 'PHYS 102': {'time': 24, 'room': 21, 'prof': 'Hughes'}, 'PHYS 103': {'time': 17, 'room': 48, 'prof': 'Hughes'}, 'PHYS 104': {'time': 4, 'room': 97, 'prof': 'Hughes'}, 'PHYS 105': {'time': 21, 'room': 97, 'prof': 'Martin'}, 'PHYS 201': {'time': 17, 'room': 91, 'prof': 'Morelli'}, 'PHYS 202': {'time': 8, 'room': 40, 'prof': 'Topper'}, 'PHYS 203': {'time': 9, 'room': 65, 'prof': 'Knobel'}, 'PHYS 204': {'time': 16, 'room': 16, 'prof': 'Fraser'}, 'PHYS 205': {'time': 28, 'room': 89, 'prof': 'Dignam'}, 'PHYS 301': {'time': 5, 'room': 45, 'prof': 'Morelli'}, 'PHYS 302': {'time': 40, 'room': 46, 'prof': 'Courteau'}, 'PHYS 303': {'time': 17, 'room': 20, 'prof': 'Dignam'}, 'PHYS 304': {'time': 29, 'room': 54, 'prof': 'Topper'}, 'PHYS 305': {'time': 40, 'room': 111, 'prof': 'Chen'}, 'PHYS 401': {'time': 24, 'room': 45, 'prof': 'Morelli'}, 'PHYS 402': {'time': 5, 'room': 106, 'prof': 'Martin'}, 'PHYS 403': {'time': 42, 'room': 44, 'prof': 'Lake'}, 'PHYS 404': {'time': 11, 'room': 106, 'prof': 'Noble'}, 'PHYS 405': {'time': 3, 'room': 30, 'prof': 'Stotz'}, 'HIST 101': {'time': 12, 'room': 99, 'prof': 'Bateman'}, 'HIST 102': {'time': 26, 'room': 34, 'prof': 'Boika'}, 'HIST 103': {'time': 40, 'room': 33, 'prof': 'Caron'}, 'HIST 104': {'time': 2, 'room': 18, 'prof': 'Levesque'}, 'HIST 105': {'time': 10, 'room': 53, 'prof': 'Levesque'}, 'HIST 201': {'time': 39, 'room': 23, 'prof': 'Collins'}, 'HIST 202': {'time': 0, 'room': 39, 'prof': 'Dougherty'}, 'HIST 203': {'time': 37, 'room': 109, 'prof': 'Castillo'}, 'HIST 204': {'time': 9, 'room': 98, 'prof': 'Hardwick'}, 'HIST 205': {'time': 16, 'room': 91, 'prof': 'Bateman'}, 'HIST 301': {'time': 1, 'room': 48, 'prof': 'Healey'}, 'HIST 302': {'time': 39, 'room': 7, 'prof': 'Dougherty'}, 'HIST 303': {'time': 29, 'room': 88, 'prof': 'Meister'}, 'HIST 304': {'time': 27, 'room': 12, 'prof': 'Hammer'}, 'HIST 305': {'time': 27, 'room': 30, 'prof': 'Meister'}, 'HIST 401': {'time': 24, 'room': 41, 'prof': 'Olinski'}, 'HIST 402': {'time': 1, 'room': 57, 'prof': 'Turner'}, 'HIST 403': {'time': 1, 'room': 45, 'prof': 'Boika'}, 'HIST 404': {'time': 9, 'room': 2, 'prof': 'Turner'}, 'HIST 405': {'time': 4, 'room': 40, 'prof': 'Walton'}, 'Fitness': 95}
sample_fitness = calc_fitness(sample_solution)
print("Sample fitness is ", sample_fitness)

Sample fitness is  [1, 10]
